In [0]:
dbutils.fs.ls('/mnt/bronze/hfpms')

[FileInfo(path='dbfs:/mnt/bronze/hfpms/card_details/', name='card_details/', size=0, modificationTime=1712080112000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/claimed_insurances/', name='claimed_insurances/', size=0, modificationTime=1712080111000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/inventory_orders/', name='inventory_orders/', size=0, modificationTime=1712080110000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/medicines/', name='medicines/', size=0, modificationTime=1712080111000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/order_details/', name='order_details/', size=0, modificationTime=1712080110000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/order_payments/', name='order_payments/', size=0, modificationTime=1712080113000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/orders/', name='orders/', size=0, modificationTime=1712080109000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/prescriptions/', name='prescriptions/', size=0, modificationTime=1712080109000),
 FileInfo(path='dbfs:/mnt/bronze/hfpms/user_ac

In [0]:
dbutils.fs.ls('/mnt/silver')

[FileInfo(path='dbfs:/mnt/silver/hfpmscard_details/', name='hfpmscard_details/', size=0, modificationTime=1712145076000),
 FileInfo(path='dbfs:/mnt/silver/hfpmsclaimed_insurances/', name='hfpmsclaimed_insurances/', size=0, modificationTime=1712145085000),
 FileInfo(path='dbfs:/mnt/silver/hfpmsinventory_orders/', name='hfpmsinventory_orders/', size=0, modificationTime=1712145094000),
 FileInfo(path='dbfs:/mnt/silver/hfpmsmedicines/', name='hfpmsmedicines/', size=0, modificationTime=1712145104000),
 FileInfo(path='dbfs:/mnt/silver/hfpmsorder_details/', name='hfpmsorder_details/', size=0, modificationTime=1712145114000),
 FileInfo(path='dbfs:/mnt/silver/hfpmsorder_payments/', name='hfpmsorder_payments/', size=0, modificationTime=1712145124000),
 FileInfo(path='dbfs:/mnt/silver/hfpmsorders/', name='hfpmsorders/', size=0, modificationTime=1712145133000),
 FileInfo(path='dbfs:/mnt/silver/hfpmsprescriptions/', name='hfpmsprescriptions/', size=0, modificationTime=1712145143000),
 FileInfo(path

In [0]:
input_path=('/mnt/bronze/hfpms/claimed_insurances/claimed_insurances.csv')

In [0]:
df=spark.read.format('csv').option("header","true").load(input_path)

In [0]:
df.head()
display(df)

id,patient_id,order_id,policy_no,claimed_amount,status,claimed_date
1,20,5,56465BHJV,50,Pending,2022-07-13 18:09:21.0000000
2,20,4,BHJF454,30,Accepted,2022-08-03 16:09:39.0000000


## Transforming date format for one table

In [0]:
from pyspark.sql.functions import date_format,from_utc_timestamp
from pyspark.sql.types import TimestampType

df=df.withColumn("claimed_date",date_format(from_utc_timestamp(df["claimed_date"].cast(TimestampType()),"UTC"),"yyyy-MM-dd"))

In [0]:
display(df)

id,patient_id,order_id,policy_no,claimed_amount,status,claimed_date
1,20,5,56465BHJV,50,Pending,2022-07-13
2,20,4,BHJF454,30,Accepted,2022-08-03


In [0]:
 #it is used to create temp view/table of dataframe result, this temp view/table will delete if spark session ends/terminates.

 df.createOrReplaceTempView("df_tempView")

In [0]:
 #we can access above temp view/table using SQL commands from Spark terminal
 spark.sql("select policy_no from df_tempView").show() 

In [0]:
%sql
-- we can access above temp view/table using SQL commands from SQL terminal
select policy_no from df_tempView  

## Transforming Date format to all tables

In [0]:
table_name=[]

for i in dbutils.fs.ls("/mnt/bronze/hfpms/"):
 table_name.append(i.name.split('/')[0])


In [0]:
table_name

['card_details',
 'claimed_insurances',
 'inventory_orders',
 'medicines',
 'order_details',
 'order_payments',
 'orders',
 'prescriptions',
 'user_activities',
 'users']

In [0]:
from pyspark.sql.functions import date_format,from_utc_timestamp
from pyspark.sql.types import TimestampType
print (table_name[5])
for i in table_name:
    path='/mnt/bronze/hfpms/'+i+'/'+i+'.csv'
    df=spark.read.format('csv').option("header","true").option("infraSchema", "true").load(path)
    column=df.columns

    for col in column:
        display(col)
        if "Date" in col or "date" in col:
            display(col)
            display(df)
            df=df.withColumn(col,date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"),"yyyy-MM-dd"))
    output_path='/mnt/silver/hfpms/'+i+'/'
    df.write.option("header","true").option("infraschema", "true").format('Delta').mode("overwrite").save(output_path)
    

   

order_payments


'id'

'payment_id'

'auth_id'

'card_brand'

...

'phone'

'address'

'role'

'status'

'registered_at'

In [0]:
input='/mnt/silver/hfpms/orders'
df=spark.read.format('delta').load(input)
display(df)

order_id,patient_id,patient_name,email_address,phone,delivery_address,order_amount,order_status,payment_status,order_date
3,20,Alexa Adams,alexa@gmail.com,0151 334 2113,2 Bromborough Village Rd Wirral Merseyside,60,pending,paid,2022-07-13
4,19,Leo Brown,leobrown@gmail.com,01642 814404,251 Acklam Rd Middlesbrough Cleveland,78,delivered,paid,2022-07-13
5,20,Andrew,andrew@gmail.com,01642 814404,251 Acklam Rd Middlesbrough Cleveland,78,pending,paid,2022-07-13
